In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df=pd.read_csv("E:\I Neuron Project\Dataset\BBC News Train.csv")
df.head()

,ArticleId,Text,Category
0,1833,worldcom ex-boss launches defence lawyers defe...,business
1,154,german business confidence slides german busin...,business
2,1101,bbc poll indicates economic gloom citizens in ...,business
3,1976,lifestyle governs mobile choice faster bett...,tech
4,917,enron bosses in $168m payout eighteen former e...,business


In [3]:
df.shape

(1490, 3)

In [6]:
df["ArticleId"].unique()

1490

In [10]:
df.drop("ArticleId",axis=1,inplace=True)

In [11]:
df.head()

,Text,Category
0,worldcom ex-boss launches defence lawyers defe...,business
1,german business confidence slides german busin...,business
2,bbc poll indicates economic gloom citizens in ...,business
3,lifestyle governs mobile choice faster bett...,tech
4,enron bosses in $168m payout eighteen former e...,business


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1490 entries, 0 to 1489
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Text      1490 non-null   object
 1   Category  1490 non-null   object
dtypes: object(2)
memory usage: 23.4+ KB


# Text Cleaning

In [13]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
ps=PorterStemmer()

In [16]:
corpus=[]
for i in range (len(df)):
    rp=re.sub('[^a-zA-Z]'," ",df['Text'][i])
    rp=rp.lower()
    rp=rp.split()
    rp=[ps.stem(word) for word in rp if not word in set(stopwords.words('english'))]
    rp=" ".join(rp)
    corpus.append(rp)

# Vectorization

In [17]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()
X=cv.fit_transform(corpus).toarray()

In [20]:
df['Category'].unique()

array(['business', 'tech', 'politics', 'sport', 'entertainment'],
      dtype=object)

In [21]:
y=df['Category'].map({'business':0,'entertainment':1,'politics':2,'sport':3,'tech':4})
y

0       0
1       0
2       0
3       4
4       0
       ..
1485    1
1486    1
1487    0
1488    4
1489    4
Name: Category, Length: 1490, dtype: int64

In [22]:
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

# Train Test Split

In [23]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=1000)

# Modeling

### Navie Bayes Classifier with default parameters

In [24]:
from sklearn.naive_bayes import MultinomialNB
model=MultinomialNB()
model.fit(X_train,y_train)

MultinomialNB()

In [28]:
from sklearn.model_selection import GridSearchCV

##HyperParamerterTuning
est=MultinomialNB()
par={'alpha':[0.1,0.3,0.5,0.7,0.9,1]}

esc=GridSearchCV(est,par,cv=5)
esc.fit(X_train,y_train)

esc.best_params_

{'alpha': 0.3}

In [29]:
model1=MultinomialNB(alpha=0.3)
model1.fit(X_train,y_train)

MultinomialNB(alpha=0.3)

In [30]:
#Prediction
ypred_test=model1.predict(X_test)
ypred_train=model1.predict(X_train)

In [33]:
ypred=model1.predict(X)

# Evalution

In [31]:
from sklearn.metrics import accuracy_score
train_accuracy=accuracy_score(y_train,ypred_train)
test_accuracy=accuracy_score(y_test,ypred_test)
print("Train accuracy : ",train_accuracy)
print("Test accuracy : ",test_accuracy)

Train accuracy :  0.9961649089165868
Test accuracy :  0.9664429530201343


In [32]:
ypred_test

array([3, 3, 1, 2, 4, 4, 1, 2, 3, 2, 2, 2, 0, 0, 4, 2, 0, 1, 3, 4, 1, 2,
       4, 1, 2, 2, 1, 1, 2, 1, 0, 4, 0, 1, 0, 3, 1, 1, 0, 0, 2, 3, 1, 4,
       0, 0, 3, 3, 0, 0, 0, 0, 4, 3, 4, 2, 0, 1, 4, 0, 0, 3, 1, 3, 0, 1,
       4, 4, 1, 4, 4, 1, 2, 0, 1, 3, 0, 2, 1, 4, 4, 2, 1, 1, 2, 4, 2, 3,
       1, 3, 2, 1, 4, 4, 0, 4, 3, 4, 2, 2, 3, 4, 2, 2, 3, 1, 0, 2, 4, 0,
       3, 1, 1, 0, 1, 2, 0, 2, 4, 1, 3, 1, 4, 0, 0, 2, 4, 0, 0, 1, 1, 2,
       1, 2, 0, 0, 4, 1, 4, 4, 1, 4, 0, 2, 4, 1, 4, 4, 3, 4, 0, 3, 3, 0,
       3, 1, 4, 4, 0, 2, 3, 4, 3, 3, 2, 0, 2, 3, 3, 1, 3, 4, 1, 4, 0, 0,
       3, 4, 2, 1, 3, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 3, 3, 3, 2, 0, 2, 1,
       2, 3, 1, 1, 1, 2, 2, 4, 0, 3, 4, 1, 4, 3, 1, 3, 1, 3, 0, 2, 2, 4,
       4, 2, 3, 3, 4, 0, 2, 4, 3, 3, 3, 3, 0, 4, 1, 2, 0, 4, 3, 3, 3, 2,
       4, 2, 1, 0, 2, 3, 4, 2, 2, 4, 1, 2, 0, 2, 1, 4, 4, 2, 0, 1, 3, 3,
       1, 2, 0, 2, 0, 4, 1, 2, 1, 0, 0, 0, 3, 4, 3, 1, 4, 2, 3, 2, 3, 0,
       3, 2, 3, 2, 1, 4, 0, 3, 4, 3, 3, 0, 1, 0, 1,

In [34]:
ypred

array([0, 0, 0, ..., 0, 4, 4], dtype=int64)

In [35]:
accuracy=accuracy_score(y,ypred)
accuracy

0.987248322147651

In [38]:
import pickle

In [41]:
pickle.dump(model1,open("NewsArtical.pkl","wb"))

In [43]:
df1=pd.read_csv("E:\I Neuron Project\Dataset\BBC News Test.csv")
df1.head()

,ArticleId,Text
0,1018,qpr keeper day heads for preston queens park r...
1,1319,software watching while you work software that...
2,1138,d arcy injury adds to ireland woe gordon d arc...
3,459,india s reliance family feud heats up the ongo...
4,1020,boro suffer morrison injury blow middlesbrough...


In [44]:
df1.shape

(735, 2)

In [45]:
corpus1=[]
for i in range (len(df1)):
    rp=re.sub('[^a-zA-Z]'," ",df1['Text'][i])
    rp=rp.lower()
    rp=rp.split()
    rp=[ps.stem(word) for word in rp if not word in set(stopwords.words('english'))]
    rp=" ".join(rp)
    corpus1.append(rp)

In [50]:
X1=cv.transform(corpus1).toarray()

In [51]:
model2=pickle.load(open("NewsArtical.pkl","rb"))

In [52]:
predict=model2.predict(X1)
predict

array([3, 4, 3, 0, 3, 3, 2, 2, 1, 0, 0, 4, 2, 4, 1, 3, 2, 4, 1, 1, 0, 2,
       3, 0, 2, 3, 0, 3, 3, 0, 2, 4, 0, 0, 3, 3, 3, 0, 1, 1, 4, 2, 1, 4,
       3, 4, 1, 0, 2, 0, 2, 0, 0, 0, 4, 2, 4, 1, 3, 4, 3, 1, 4, 2, 1, 1,
       3, 4, 3, 3, 4, 3, 0, 2, 4, 3, 4, 4, 4, 1, 2, 3, 1, 1, 0, 1, 0, 1,
       0, 4, 2, 2, 3, 4, 3, 3, 3, 3, 3, 3, 2, 3, 2, 1, 0, 3, 2, 3, 2, 1,
       3, 0, 1, 3, 2, 3, 2, 3, 2, 0, 1, 0, 1, 1, 4, 3, 0, 1, 0, 1, 0, 2,
       2, 4, 0, 0, 2, 4, 1, 3, 0, 4, 3, 1, 2, 3, 3, 1, 1, 4, 0, 4, 2, 1,
       3, 3, 3, 3, 1, 4, 0, 4, 0, 4, 0, 4, 1, 4, 4, 2, 0, 2, 0, 0, 1, 2,
       4, 0, 0, 4, 3, 2, 3, 2, 4, 4, 2, 0, 2, 4, 2, 0, 1, 3, 4, 4, 0, 4,
       2, 0, 3, 2, 0, 1, 0, 0, 3, 4, 0, 3, 1, 1, 3, 1, 3, 4, 2, 1, 3, 1,
       3, 1, 2, 0, 4, 1, 0, 2, 0, 4, 0, 3, 2, 2, 2, 2, 3, 0, 2, 2, 3, 2,
       0, 3, 4, 0, 2, 0, 2, 0, 0, 3, 4, 2, 4, 4, 1, 4, 3, 3, 4, 3, 3, 3,
       1, 3, 2, 4, 4, 3, 0, 3, 0, 3, 1, 0, 0, 1, 2, 0, 3, 3, 4, 3, 3, 1,
       0, 3, 4, 2, 1, 0, 0, 2, 3, 1, 2, 0, 3, 3, 4,

In [49]:
predict.shape

(1490,)

In [53]:
sub=pd.DataFrame({"ArticleId":df1["ArticleId"],"text":df1["Text"],"Final_prediction":predict})
sub

,ArticleId,text,Final_prediction
0,1018,qpr keeper day heads for preston queens park r...,3
1,1319,software watching while you work software that...,4
2,1138,d arcy injury adds to ireland woe gordon d arc...,3
3,459,india s reliance family feud heats up the ongo...,0
4,1020,boro suffer morrison injury blow middlesbrough...,3
...,...,...,...
730,1923,eu to probe alitalia state aid the european ...,0
731,373,u2 to play at grammy awards show irish rock ba...,1
732,1704,sport betting rules in spotlight a group of mp...,2
733,206,alfa romeos to get gm engines fiat is to sto...,0


In [ ]:
sub['Final_prediction']=sub['Final_prediction'].map({0:'business',1:'entertainment',2:'politics',3:'sport',4:'tech'})
sub